In [9]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels import RandomEffects
from sklearn.preprocessing import MinMaxScaler

In [10]:
panel_data = pd.read_csv('Data/final/Rental_5year.csv')

panel_data.head()

,index,borough,price_long_rents,count_long_rents,year,price_airbnb,borough_area,count_airbnb,density_airbnb,code
0,2023,Barking and Dagenham,1350.025316,790,2023,245.840426,36107810,564,1.561989,1
1,2023,Barnet,1701.178082,2190,2023,160.523344,86748314,2249,2.592558,2
2,2023,Bexley,1399.245283,530,2023,95.046465,60580233,495,0.817098,3
3,2023,Brent,1628.101695,590,2023,179.079197,43232640,2740,6.337804,4
4,2023,Bromley,1499.208696,2300,2023,105.972798,150134858,772,0.514204,5


In [11]:
panel_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             165 non-null    int64  
 1   borough           165 non-null    object 
 2   price_long_rents  165 non-null    float64
 3   count_long_rents  165 non-null    int64  
 4   year              165 non-null    int64  
 5   price_airbnb      165 non-null    float64
 6   borough_area      165 non-null    int64  
 7   count_airbnb      165 non-null    int64  
 8   density_airbnb    165 non-null    float64
 9   code              165 non-null    int64  
dtypes: float64(3), int64(6), object(1)
memory usage: 13.0+ KB


In [12]:
panel_data = panel_data.set_index(["index", "code"])

In [15]:
# 选择自变量和因变量

independent_vars = [
    "price_airbnb",
    "density_airbnb",
]
dependent_var = "price_long_rents"  # 之前计算的综合指数

# 准备模型的因变量和自变量
Y = panel_data[dependent_var]
X = panel_data[independent_vars]

# 添加常数项
X = sm.add_constant(X)

In [16]:
# 构建随机效应模型
model = RandomEffects(Y, X)
results = model.fit()

# 输出模型结果
print(results)

                        RandomEffects Estimation Summary                        
Dep. Variable:       price_long_rents   R-squared:                        0.8532
Estimator:              RandomEffects   R-squared (Between):              0.7839
No. Observations:                 165   R-squared (Within):               0.8551
Date:                Thu, Dec 07 2023   R-squared (Overall):              0.8525
Time:                        18:35:16   Log-likelihood                   -1072.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      470.84
Entities:                           5   P-value                           0.0000
Avg Obs:                       33.000   Distribution:                   F(2,162)
Min Obs:                       33.000                                           
Max Obs:                       33.000   F-statistic (robust):             470.84
                            

In [17]:
from linearmodels.panel import PanelOLS

# 估计固定效应模型
model = PanelOLS(Y, X, entity_effects=True)
results = model.fit()

# 输出固定效应模型的结果
print(results)


                          PanelOLS Estimation Summary                           
Dep. Variable:       price_long_rents   R-squared:                        0.8556
Estimator:                   PanelOLS   R-squared (Between):              0.7423
No. Observations:                 165   R-squared (Within):               0.8556
Date:                Thu, Dec 07 2023   R-squared (Overall):              0.8515
Time:                        18:35:28   Log-likelihood                   -1069.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      467.98
Entities:                           5   P-value                           0.0000
Avg Obs:                       33.000   Distribution:                   F(2,158)
Min Obs:                       33.000                                           
Max Obs:                       33.000   F-statistic (robust):             467.98
                            

In [90]:
panel_data_diff = panel_data['price_airbnb'].diff().dropna()

In [53]:
panel_data.head()

borough  price_long_rents  count_long_rents  year  \
index code                                                                   
2023  1     Barking and Dagenham       1350.025316               790  2023   
      2                   Barnet       1701.178082              2190  2023   
      3                   Bexley       1399.245283               530  2023   
      4                    Brent       1628.101695               590  2023   
      5                  Bromley       1499.208696              2300  2023   

            price_airbnb  borough_area  count_airbnb  density_airbnb  
index code                                                            
2023  1       245.840426      36107810           564        1.561989  
      2       160.523344      86748314          2249        2.592558  
      3        95.046465      60580233           495        0.817098  
      4       179.079197      43232640          2740        6.337804  
      5       105.972798     150134858           772        0.514204

In [91]:
from statsmodels.tsa.stattools import adfuller

# 进行 ADF 单位根检验
#result = adfuller(panel_data['price_long_rents'])
result = adfuller(panel_data['price_airbnb'])
#result = adfuller(panel_data['density_airbnb'])

# 输出 ADF 检验的结果
print('ADF单位根检验结果:')
print(f'ADF统计量: {result[0]}')
print(f'P值: {result[1]}')
print(f'滞后阶数: {result[2]}')
print(f'观察值个数: {result[3]}')
print(f'临界值: {result[4]}')
print(f'1%显著性水平的临界值: {result[4]["1%"]}')
print(f'5%显著性水平的临界值: {result[4]["5%"]}')
print(f'10%显著性水平的临界值: {result[4]["10%"]}')



ADF单位根检验结果:
ADF统计量: -9.328082621088178
P值: 9.490544419785538e-16
滞后阶数: 11
观察值个数: 152
临界值: {'1%': -3.474120870218417, '5%': -2.880749791423677, '10%': -2.5770126333102494}
1%显著性水平的临界值: -3.474120870218417
5%显著性水平的临界值: -2.880749791423677
10%显著性水平的临界值: -2.5770126333102494


In [95]:
from statsmodels.tsa.stattools import coint

# 进行协整关系检验
result = coint(panel_data['density_airbnb'], panel_data['price_airbnb'])

# 输出协整关系检验的结果
print('协整关系检验结果:')
print(f'Cointegration statistic: {result[0]}')
print(f'P-value: {result[1]}')
print(f'Critical values: {result[2]}')


协整关系检验结果:
Cointegration statistic: -1.252563099934649
P-value: 0.8435943093379856
Critical values: [-3.96446642 -3.37364039 -3.07041211]


In [66]:
#BP检验
from statsmodels.compat import lzip
from statsmodels.stats.diagnostic import het_breuschpagan

# 估计随机效应模型
model = sm.OLS(Y, X)
results = model.fit()

# 进行Breusch-Pagan检验
bp_test = het_breuschpagan(results.resid, X)

# 输出检验结果
labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']
print(lzip(labels, bp_test))


[('LM Statistic', 13.287780815293392), ('LM-Test p-value', 0.0013019522533443803), ('F-Statistic', 7.094420290091326), ('F-Test p-value', 0.001112925174479028)]


In [77]:
#Hausman检验
from scipy.stats import chi2

model_fe = PanelOLS(Y, X, entity_effects=True).fit()
model_re = RandomEffects(Y, X).fit()

# Calculate the difference in coefficients
coeff_diff = model_fe.params - model_re.params

# Calculate the variance-covariance matrix of the difference
vcov_diff = model_fe.cov - model_re.cov

# Calculate the Hausman test statistic
hausman_statistic = coeff_diff @ np.linalg.inv(vcov_diff) @ coeff_diff

# Calculate degrees of freedom
df = len(coeff_diff)

# Calculate p-value using chi-square distribution
p_value = 1 - chi2.cdf(hausman_statistic, df)

# Output Hausman test result
print(f'Hausman Statistic: {hausman_statistic}')
print(f'Degrees of Freedom: {df}')
print(f'P-value: {p_value}')

Hausman Statistic: 1.562698901347319
Degrees of Freedom: 3
P-value: 0.6678768501827694
